### Layered LightGBM Ensemble: Sampling from Clusters, Time-Sliced Training and Ridge Stacking

In this notebook, I build on the hierarchical clustering performed on the masked features in the feature_engineering notebook. Instead of aggregating features within each cluster, I sample directly from the clusters. Specifically, in each round I select one feature from each cluster without replacement, ensuring that no two features from the same cluster appear in a given subset. I continue this procedure until all features are sampled. This procedure is repeated until all features have been sampled.

I then apply the same time-slicing procedure, splitting the training data into 5 subsamples. On each subsample, I train multiple boosted trees using the sampled feature subsets, resulting in total of 60 boosted trees. To guard against overfitting in this noisy setting, I constrain the search space for model hyperparameters (e.g., lower ranges for bagging and feature fractions, and fewer boosted trees), encouraging simpler models that generalize better. Finally, to ensemble their outputs, I train a ridge regression model on the predictions from all trees.

In [1]:
# libraries and settings
import gc
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.stats import pearsonr
from sklearn.linear_model import Ridge
import lightgbm as lgb
import optuna
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_parquet('../data/input/train.parquet')
df_test = pd.read_parquet('../data/input/test.parquet')

#### Sampling Features
After the feature sampling procedure is complete, feature samples that have less than 10 features are dropped. Moreover, the orderbook features is added to all masked feature subsets to be used in the training procedure. In the end, Iend up with 12 feature subsets.

In [3]:
# Clustering the features
X_cols = df_train.columns[df_train.columns.str.startswith('X')]
clusters = pd.Series(pd.read_csv('../data/intermediate/clusters.csv', index_col = 0).iloc[:,0])

# shuffle the features
clusters = clusters.sample(frac = 1, random_state = 42)

# sampling teh clusters
clusters_dict = defaultdict(list)

for c in np.unique(clusters):
    X = clusters[clusters == c]
    clusters_dict[c] = X.index.tolist()

total = 0
features_list = []
n_features = len(clusters)

while total < n_features:
    features = []
    for c in clusters_dict.keys():
        if len(clusters_dict[c]) > 0:
            features.append(clusters_dict[c][-1])
            clusters_dict[c].pop()
            total += 1
    features_list.append(features)
    features = list([])
    
while len(features_list[-1]) < 10:
    features_list.pop()

for i in range(len(features_list)):
    features_list[i] = ['bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume'] + features_list[i]

#### Ensemble Prediction Model
I design a class to pipeline the tuning and training procedures. First, the training and validation datasets are split into 5 subsamples by selecting every 5th row (after sorting teh data based on their timestamps) into each subsample. For each subsample, I train boosted trees on the corresponding training data and feature subsets, tuning them on their paired validation subsample using the correlation metric. The correlation metric is chosen because it was the evaluation metric used in the competition.

Note that because the data is highly noisy, I restrict the search space for the bagging fraction and feature fraction to the lower ranges, [0.2, 0.6] and [0.1,0.5], respectively. Likewise, the number of boosted trees is tuned within the range of 200 to 500, which is also on the lower end. These choices are intended to promote better generalization in the presence of noisy inputs.

Once all 60 tuned trees are trained, I generate predictions from them on the full training and validation datasets. These stacked predictions form the inputs to a ridge regression model, which is trained on the training predictions. The ridge regularization parameter is tuned based on the correlation metric evaluated on the validation predictions.

Finally, after identifying the optimal ridge parameter, I retrain all boosted trees with their tuned hyperparameters on the full dataset (including the validation sample). I then refit the ridge regression using the optimized regularization parameter on the stacked predictions from the full data. This final ensemble model is used to produce predictions on the test dataset.

In [5]:
""" **************** ENSEMBLE MODEL ****************** """
class LightGBMTimeSeriesEnsemble:
    def __init__(self, n_slices, n_trials, feature_sets, alphas=None,
                 metric="l2", random_seed=42):
        """
        n_slices  : number of time slices (e.g. in our case 20)
        n_trials  : Optuna trials per learner
        feature_sets: sets of features that are each separately used for training trees.
        alphas    : candidate ridge regularization strengths
        metric    : LightGBM eval metric (for early stopping)
        """
        self.n_slices = n_slices
        self.n_trials = n_trials
        self.feature_sets = feature_sets
        self.metric = metric
        self.alphas = alphas if alphas is not None else np.logspace(-2, 5, 30)
        self.seed = random_seed

        self.best_params_list = []
        self.best_iterations = []
        self.models = []
        self.ridge = None
        self.best_alpha = None

    def _optuna_objective(self, trial, X_tr, y_tr, X_val, y_val):
        params = {
            'objective': 'regression',
            'metric': 'rmse',  
            'verbosity': -1,
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 16, 64),
            'max_depth': trial.suggest_int('max_depth', 2, 6),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 0.5),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.2, 0.6),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
            'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
            'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
            'num_boost_round': trial.suggest_int('num_boost_round', 200,500),
            'early_stopping_rounds': 50
        }
        model = lgb.train(
            params,
            lgb.Dataset(X_tr, y_tr),
            valid_sets=[lgb.Dataset(X_val, y_val)]
        )
        pred = model.predict(X_val)
        corr, _ = pearsonr(pred, y_val)
        return -corr

    def fit(self, X_train, y_train, X_val, y_val):
        """
        1) Fit each base model on X_train[i::n] versus y_train[i::n] with Optuna tuning.
        Next steps are carried out in the fit_ridge function:
        2) Make predictions of each base model on *full* X_train => (n_samples_train x n_slices)
        3) Ridge-fit on (train_preds, y_train)
        4) Select alpha by evaluating ridge on full X_val predictions
        """
        self.models = []
        self.best_params_list = []
        self.best_iterations = []

        for j in range(len(self.feature_sets)):
            for i in range(self.n_slices):
                print(f'Training on feature set {j+1}/{len(self.feature_sets)}, model {i+1}/{self.n_slices}')
                
                Xt = X_train.iloc[i::self.n_slices][self.feature_sets[j]]
                yt = y_train.iloc[i::self.n_slices]
                Xv = X_val.iloc[i::self.n_slices][self.feature_sets[j]]
                yv = y_val.iloc[i::self.n_slices]
    
                # Tune
                sampler = optuna.samplers.TPESampler(seed=self.seed)
                study = optuna.create_study(direction='minimize', sampler=sampler)
                study.optimize(
                    lambda trial: self._optuna_objective(trial, Xt, yt, Xv, yv),
                    n_trials=self.n_trials,
                )
                best_params = study.best_params
                best_params['seed'] = self.seed
                
    
                # Train model with best params
                model = lgb.train(
                    {
                    **best_params, 
                    'objective':'regression',
                    'metric':self.metric,
                    'verbosity':-1, 
                    'early_stopping_rounds': 50
                    },
                    lgb.Dataset(Xt, yt),
                    valid_sets=[lgb.Dataset(Xv, yv)]
                )
                best_params['num_boost_round'] = model.best_iteration
                self.best_params_list.append(best_params)
                self.best_iterations.append(model.best_iteration)
                self.models.append(model)

        self.fit_ridge(X_train, y_train, X_val, y_val, self.alphas)
        
        
    def fit_ridge(self, X_train, y_train, X_val, y_val, alphas):
        """
        1) Make predictions of each base model on *full* X_train => (n_samples_train x n_slices)
        2) Ridge-fit on (train_preds, y_train)
        3) Select alpha by evaluating ridge on full X_val predictions
        """
        train_preds = []
        val_preds   = []
        idx = 0

         # --- predictions on full training and validation set ---
        for subset in self.feature_sets:
            for _ in range(self.n_slices):
                m  = self.models[idx]
                it = self.best_iterations[idx]
                train_preds.append(m.predict(X_train[subset], num_iteration=it))
                val_preds.append(m.predict(X_val[subset],   num_iteration=it))
                idx += 1
        
        train_stack = np.column_stack(train_preds)
        val_stack   = np.column_stack(val_preds)
       
        self.alphas = alphas
        best_corr = -np.inf
        for a in self.alphas:
            r = Ridge(alpha=a)
            r.fit(train_stack, y_train)
            corr, _ = pearsonr(r.predict(val_stack), y_val)
            print(f' ridge alpha: {a:.5f}, val corr: {corr:.5f}')
            if corr > best_corr:
                best_corr, self.best_alpha, self.ridge = corr, a, r

        print(f'Best ridge alpha: {self.best_alpha:.5f}, val corr: {best_corr:.5f}')
    
    
    def refit_full(self, X_full, y_full):
        """
        Retrain base models and the ridge regression model on full data using best params and alpha.
        """
        self.models = []
        idx = 0
        for subset in self.feature_sets:
            for _ in range(self.n_slices):
                print(f'Refitting model {idx+1}/{self.n_slices*len(self.feature_sets)} on full data...')
                params  = self.best_params_list[idx]
                model = lgb.train(
                    {
                    **params,  
                    'objective':'regression',
                    'metric':self.metric,
                    'verbosity':-1
                    },
                    lgb.Dataset(X_full[subset], y_full)
                )
                self.models.append(model)
                idx += 1
        
        preds = []
        idx = 0
         # --- predictions on full dataset
        for subset in self.feature_sets:
            for _ in range(self.n_slices):
                m  = self.models[idx]
                it = self.best_iterations[idx]
                preds.append(m.predict(X_full[subset], num_iteration=it))
                idx += 1
                
        full_stack = stack = np.column_stack(preds)       
        self.ridge = Ridge(alpha=self.best_alpha)
        self.ridge.fit(full_stack, y_full)

    def predict(self, X):
        """
        Predict stacked output on full dataset X
        """
        preds = []
        idx = 0
        for subset in self.feature_sets:
            for _ in range(self.n_slices):
                m  = self.models[idx]
                it = self.best_iterations[idx]
                preds.append(m.predict(X[subset], num_iteration=it))
                idx += 1
        stack = np.column_stack(preds)
        
        return self.ridge.predict(stack)



Due to the time-series nature of the data, I sort observations by their timestamps to prevent any look-ahead bias. The first 80% of the data is used for training, while the remaining 20% is reserved for validation.

In [6]:
df_train.sort_index(inplace = True)
n_train = round(df_train.shape[0]*0.8)
X_train = df_train.iloc[:n_train].drop(columns = ['label'])
X_val = df_train.iloc[n_train:].drop(columns = ['label'])
y_train = df_train.iloc[:n_train]['label'] 
y_val = df_train.iloc[n_train:]['label'] 

ensemble = LightGBMTimeSeriesEnsemble(n_slices=5, n_trials=50, feature_sets=features_list)
ensemble.fit(X_train, y_train, X_val, y_val)

[I 2025-08-19 14:47:32,047] A new study created in memory with name: no-name-94a59bad-11a4-4401-87c3-51df3e531bc1


Training on feature set 1/12, model 1/5


[I 2025-08-19 14:47:33,925] Trial 0 finished with value: -0.08080226745370152 and parameters: {'learning_rate': 0.015355286838886862, 'num_leaves': 62, 'max_depth': 5, 'min_data_in_leaf': 64, 'feature_fraction': 0.1624074561769746, 'bagging_fraction': 0.26239780813448105, 'bagging_freq': 1, 'lambda_l1': 4.330880728874676, 'lambda_l2': 3.005575058716044, 'num_boost_round': 413}. Best is trial 0 with value: -0.08080226745370152.
[I 2025-08-19 14:47:35,966] Trial 1 finished with value: -0.0829087427784682 and parameters: {'learning_rate': 0.005318033256270142, 'num_leaves': 63, 'max_depth': 6, 'min_data_in_leaf': 29, 'feature_fraction': 0.17272998688284025, 'bagging_fraction': 0.27336180394137355, 'bagging_freq': 4, 'lambda_l1': 2.6237821581611893, 'lambda_l2': 2.1597250932105787, 'num_boost_round': 287}. Best is trial 1 with value: -0.0829087427784682.
[I 2025-08-19 14:47:37,512] Trial 2 finished with value: -0.08354512870665791 and parameters: {'learning_rate': 0.03126143958203108, 'num

Training on feature set 1/12, model 2/5


[I 2025-08-19 14:49:10,237] A new study created in memory with name: no-name-2a66fce1-b68f-4873-af17-66419fd7296d
[I 2025-08-19 14:49:11,710] Trial 0 finished with value: -0.07518809983599434 and parameters: {'learning_rate': 0.015355286838886862, 'num_leaves': 62, 'max_depth': 5, 'min_data_in_leaf': 64, 'feature_fraction': 0.1624074561769746, 'bagging_fraction': 0.26239780813448105, 'bagging_freq': 1, 'lambda_l1': 4.330880728874676, 'lambda_l2': 3.005575058716044, 'num_boost_round': 413}. Best is trial 0 with value: -0.07518809983599434.
[I 2025-08-19 14:49:14,105] Trial 1 finished with value: -0.09776442774770872 and parameters: {'learning_rate': 0.005318033256270142, 'num_leaves': 63, 'max_depth': 6, 'min_data_in_leaf': 29, 'feature_fraction': 0.17272998688284025, 'bagging_fraction': 0.27336180394137355, 'bagging_freq': 4, 'lambda_l1': 2.6237821581611893, 'lambda_l2': 2.1597250932105787, 'num_boost_round': 287}. Best is trial 1 with value: -0.09776442774770872.
[I 2025-08-19 14:49:1

...

Training on feature set 12/12, model 4/5


[I 2025-08-19 15:22:03,496] Trial 0 finished with value: -0.03140718341285065 and parameters: {'learning_rate': 0.015355286838886862, 'num_leaves': 62, 'max_depth': 5, 'min_data_in_leaf': 64, 'feature_fraction': 0.1624074561769746, 'bagging_fraction': 0.26239780813448105, 'bagging_freq': 1, 'lambda_l1': 4.330880728874676, 'lambda_l2': 3.005575058716044, 'num_boost_round': 413}. Best is trial 0 with value: -0.03140718341285065.
[I 2025-08-19 15:22:03,983] Trial 1 finished with value: -0.04586556494130817 and parameters: {'learning_rate': 0.005318033256270142, 'num_leaves': 63, 'max_depth': 6, 'min_data_in_leaf': 29, 'feature_fraction': 0.17272998688284025, 'bagging_fraction': 0.27336180394137355, 'bagging_freq': 4, 'lambda_l1': 2.6237821581611893, 'lambda_l2': 2.1597250932105787, 'num_boost_round': 287}. Best is trial 1 with value: -0.04586556494130817.
[I 2025-08-19 15:22:04,175] Trial 2 finished with value: -0.05599326166323471 and parameters: {'learning_rate': 0.03126143958203108, 'n

Training on feature set 12/12, model 5/5


[I 2025-08-19 15:22:15,709] Trial 0 finished with value: -0.035482624352066464 and parameters: {'learning_rate': 0.015355286838886862, 'num_leaves': 62, 'max_depth': 5, 'min_data_in_leaf': 64, 'feature_fraction': 0.1624074561769746, 'bagging_fraction': 0.26239780813448105, 'bagging_freq': 1, 'lambda_l1': 4.330880728874676, 'lambda_l2': 3.005575058716044, 'num_boost_round': 413}. Best is trial 0 with value: -0.035482624352066464.
[I 2025-08-19 15:22:16,018] Trial 1 finished with value: -0.02281703646965043 and parameters: {'learning_rate': 0.005318033256270142, 'num_leaves': 63, 'max_depth': 6, 'min_data_in_leaf': 29, 'feature_fraction': 0.17272998688284025, 'bagging_fraction': 0.27336180394137355, 'bagging_freq': 4, 'lambda_l1': 2.6237821581611893, 'lambda_l2': 2.1597250932105787, 'num_boost_round': 287}. Best is trial 0 with value: -0.035482624352066464.
[I 2025-08-19 15:22:16,178] Trial 2 finished with value: -0.03693647326672452 and parameters: {'learning_rate': 0.03126143958203108,

 ridge alpha: 0.01000, val corr: 0.07434
 ridge alpha: 0.01743, val corr: 0.07434
 ridge alpha: 0.03039, val corr: 0.07434
 ridge alpha: 0.05298, val corr: 0.07433
 ridge alpha: 0.09237, val corr: 0.07432
 ridge alpha: 0.16103, val corr: 0.07430
 ridge alpha: 0.28072, val corr: 0.07427
 ridge alpha: 0.48939, val corr: 0.07422
 ridge alpha: 0.85317, val corr: 0.07415
 ridge alpha: 1.48735, val corr: 0.07406
 ridge alpha: 2.59294, val corr: 0.07397
 ridge alpha: 4.52035, val corr: 0.07398
 ridge alpha: 7.88046, val corr: 0.07427
 ridge alpha: 13.73824, val corr: 0.07512
 ridge alpha: 23.95027, val corr: 0.07686
 ridge alpha: 41.75319, val corr: 0.07981
 ridge alpha: 72.78954, val corr: 0.08418
 ridge alpha: 126.89610, val corr: 0.09004
 ridge alpha: 221.22163, val corr: 0.09722
 ridge alpha: 385.66204, val corr: 0.10525
 ridge alpha: 672.33575, val corr: 0.11324
 ridge alpha: 1172.10230, val corr: 0.12005
 ridge alpha: 2043.35972, val corr: 0.12481
 ridge alpha: 3562.24789, val corr: 0.1

In [9]:
# further fine-tuning the ridge regression regularization parameter
ensemble.fit_ridge(X_train, y_train, X_val, y_val, range(5000, 9000, 200))

 ridge alpha: 5000.00000, val corr: 0.12781
 ridge alpha: 5200.00000, val corr: 0.12784
 ridge alpha: 5400.00000, val corr: 0.12785
 ridge alpha: 5600.00000, val corr: 0.12786
 ridge alpha: 5800.00000, val corr: 0.12786
 ridge alpha: 6000.00000, val corr: 0.12786
 ridge alpha: 6200.00000, val corr: 0.12785
 ridge alpha: 6400.00000, val corr: 0.12784
 ridge alpha: 6600.00000, val corr: 0.12783
 ridge alpha: 6800.00000, val corr: 0.12781
 ridge alpha: 7000.00000, val corr: 0.12779
 ridge alpha: 7200.00000, val corr: 0.12777
 ridge alpha: 7400.00000, val corr: 0.12775
 ridge alpha: 7600.00000, val corr: 0.12773
 ridge alpha: 7800.00000, val corr: 0.12770
 ridge alpha: 8000.00000, val corr: 0.12767
 ridge alpha: 8200.00000, val corr: 0.12765
 ridge alpha: 8400.00000, val corr: 0.12762
 ridge alpha: 8600.00000, val corr: 0.12759
 ridge alpha: 8800.00000, val corr: 0.12756
Best ridge alpha: 5800.00000, val corr: 0.12786


The final validation score is 0.12786, which is an improvement over the previous model that used aggregated features from each cluster.

In [10]:
ensemble.refit_full(df_train.drop(columns='label'), df_train['label'])

y_test_pred = ensemble.predict(df_test)
y_test_pred_pd = pd.Series(y_test_pred, name = 'prediction')
y_test_pred_pd.index = df_test.index
y_test_pred_pd.to_csv('../data/output/submission6.csv')

Refitting model 1/60 on full data...
Refitting model 2/60 on full data...
Refitting model 3/60 on full data...
Refitting model 4/60 on full data...
Refitting model 5/60 on full data...
Refitting model 6/60 on full data...
Refitting model 7/60 on full data...
Refitting model 8/60 on full data...
Refitting model 9/60 on full data...
Refitting model 10/60 on full data...
Refitting model 11/60 on full data...
Refitting model 12/60 on full data...
Refitting model 13/60 on full data...
Refitting model 14/60 on full data...
Refitting model 15/60 on full data...
Refitting model 16/60 on full data...
Refitting model 17/60 on full data...
Refitting model 18/60 on full data...
Refitting model 19/60 on full data...
Refitting model 20/60 on full data...
Refitting model 21/60 on full data...
Refitting model 22/60 on full data...
Refitting model 23/60 on full data...
Refitting model 24/60 on full data...
Refitting model 25/60 on full data...
Refitting model 26/60 on full data...
Refitting model 27/60